In [ ]:
import xml.etree.ElementTree as et 
from gensim import corpora
from collections import defaultdict
import glob 

#Dictionary for all award ID's and Abstracts 
global awards
awards=dict()

#Array storage for Award IDs and Abstracts 
awardabs_ar=["Null"]
aID_ar=["Null"]

#Counts The number of items found with an Abstract vs without 
without = 0
T_with = 0 
grandtotal = 0

#Used as a condition for later statement 
x = 0

#Function Declaration 
def addIDandAbs(ID,abstract):
    for i in awards:
        #Checks for duplicates 
        if i == ID:
            print("Error")
            return
    #Store ID and Abstract
    awards[ID] = abstract
    awardabs.append(abstract)
    aID_ar.append(ID)
    
for items in glob.glob("201*/*.xml"): #Iterate through all xml files in the directory/file name given
    #Open files
    item = open(items)
    #Store Data
    str_data = item.read()
    #Convert data to text 
    data = et.fromstring(str_data)
    #store the current name of the institution 
    clemson = data.find("Award").find("Institution").find("Name").text
    #Check if the name matches Clemson University 
    if clemson=="Clemson University" :
        #Add award ID and Abstract to the dictionary awards
        #addIDandAbs(data.find("Award").find("AwardID").text,data.find("Award").find("AbstractNarration").text)
        ID = data.find("Award").find("AwardID").text
        abstract = data.find("Award").find("AbstractNarration").text
        awards[ID]=abstract
        #Keep count of the number of items found 
        T_with = T_with + 1
    else:
        #Optional Keep count of the number of items not found 
        without = without + 1
    #Optional Total Articles:
    grandtotal = grandtotal + 1
#Store Data into arrays from the library 
for i,v in awards.items():
    #Stores the first abstract and key into the first part of the array so append will see that the list is not empty.
    if x == 0:
        awardabs_ar[0]=v
        aID_ar[0]=i
    else:
        awardabs_ar.append(v)
        aID_ar.append(i)
    x = x + 1
#Optional print code for data in first part of array
#for l in range(T_with):
    #print("Award ID:\n"+aID_ar[0],"\n\nAward Abstract:\n"+awardabs_ar[0]+"\n")
    
#remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist]
          for document in awardabs_ar]

# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1]
          for text in texts]